Arbol elemental con libreria  rpart

In [1]:
require("data.table")
require("rpart")

Loading required package: data.table

Loading required package: rpart



In [2]:
#Aqui se debe poner la carpeta de la computadora local
setwd("E:/Archivo/EconFin")  #Establezco el Working Directory

cargo los datos de 202009 que es donde voy a ENTRENAR el modelo

In [13]:
dtrain  <- fread("./datasetsOri/paquete_premium_202009.csv")

Cargo los datos de 202111, para luego aplicarle el modelo

In [14]:
dapply  <- fread("./datasetsOri/paquete_premium_202011.csv")

In [15]:
View(head(dtrain,10))

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,...,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
4572294,202009,1,0,1,46,89,2183.00,9702.36,1082.25,...,7,0.00,-55893.73,0.00,2691,11705.75,5,0,9558.9,CONTINUA
4572366,202009,1,0,1,48,198,11875.55,43936.81,2076.22,...,0,0.00,-20021.04,0.00,3363,3079.08,5,0,17667.0,CONTINUA
4572633,202009,1,0,1,56,124,25658.55,525024.30,2936.75,...,0,0.00,-61794.90,0.00,3706,47735.01,14,0,3486.6,CONTINUA
4572728,202009,1,0,1,56,272,3420.31,2919.85,1054.49,...,0,0.00,-38565.16,0.00,4952,18471.29,9,0,1251.9,CONTINUA
4572765,202009,1,0,1,49,299,4212.94,28345.98,1999.98,...,0,0.00,-22495.71,23.65,2066,0.00,0,0,1404.0,CONTINUA
4572841,202009,1,0,1,60,369,8004.91,139974.06,625.83,...,0,0.00,-66924.00,0.00,7651,92439.73,13,0,35100.0,BAJA+2
4572913,202009,1,0,1,45,269,10030.36,98933.73,853.65,...,0,0.00,-11009.95,0.00,1940,846.16,1,0,70.2,CONTINUA
4572976,202009,1,0,1,79,316,1441.13,59200.29,1775.88,...,0,0.00,-2671.55,0.00,7398,12548.97,7,0,503.1,CONTINUA
4573161,202009,1,0,1,61,274,-125.98,57574.50,63.63,...,7,256.93,-256.93,0.00,7451,253.89,3,0,11.7,CONTINUA


In [16]:
## LIMPIEZA DEL DATASET

# Columnas a eliminar del dataset por Data Drifting de Densidad
dtrain[,c("internet","mcuenta_corriente", "mcaja_ahorro_adicional",
                 "mcaja_ahorro_dolares", "mprestamos_prendarios",
                "mprestamos_hipotecarios", "mplazo_fijo_dolares", "mplazo_fijo_pesos",
                "minversion1_pesos", "minversion1_dolares", "mtarjeta_visa_descuentos",
                "mtarjeta_master_descuentos", "tmobile_app", "cmobile_app_trx", 
                "Master_madelantopesos", "Master_madelantodolares", "Visa_msaldodolares"
  ):=NULL]

dapply[,c("internet","mcuenta_corriente", "mcaja_ahorro_adicional",
           "mcaja_ahorro_dolares", "mprestamos_prendarios",
           "mprestamos_hipotecarios", "mplazo_fijo_dolares", "mplazo_fijo_pesos",
           "minversion1_pesos", "minversion1_dolares", "mtarjeta_visa_descuentos",
           "mtarjeta_master_descuentos", "tmobile_app", "cmobile_app_trx", 
           "Master_madelantopesos", "Master_madelantodolares", "Visa_msaldodolares"
  ):=NULL]

#View(dataset[,c("mactivos_margen_rank")])

# Columnas a rankear por Data Drifting de Delta 
campos_rankear = c("mactivos_margen", "mpayroll", "matm_other", "Master_fultimo_cierre", 
                   "Visa_mfinanciacion_limite", "Master_Finiciomora", "Visa_mconsumosdolares")

for( campo in campos_rankear )
{
  campo_nuevo_name = paste(campo, "_rank", sep = "")
  
  campo_rank = frankv(dtrain, campo,
                                order = 1L,
                                na.last = "keep",
                                ties.method="min")
  dtrain = cbind(dtrain, campo_nuevo_name = campo_rank)
  setnames(dtrain, "campo_nuevo_name", campo_nuevo_name)
  dtrain[,c(campo):=NULL]

  campo_rank = frankv(dapply, campo,
                      order = 1L,
                      na.last = "keep",
                      ties.method="min")
  dapply = cbind(dapply, campo_nuevo_name = campo_rank)
  setnames(dapply, "campo_nuevo_name", campo_nuevo_name)
  dapply[,c(campo):=NULL]
  
  print(campo_nuevo_name)
}


[1] "mactivos_margen_rank"
[1] "mpayroll_rank"
[1] "matm_other_rank"
[1] "Master_fultimo_cierre_rank"
[1] "Visa_mfinanciacion_limite_rank"
[1] "Master_Finiciomora_rank"
[1] "Visa_mconsumosdolares_rank"


In [17]:
# Function weights
positiveWeight = (100000/2)
negativeWeight = 1250
modelWeights <- ifelse(dtrain$clase_ternaria== 'BAJA+2', positiveWeight, negativeWeight)

In [18]:
modelo  <- rpart("clase_ternaria ~ .",
                 data = dtrain,
                 xval=0,
                 cp =        0,#-0.26099, #0
                 minsplit=  4,#1531, #4
                 minbucket=  4,#466, #4
                 maxdepth=   14,#8, #14
                 weights = modelWeights
                )

In [19]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

In [20]:
dapply[ , prob_baja2 := prediccion[, "BAJA+2"] ]
dapply[ , Predicted  := as.numeric(prob_baja2 > 0.5) ]

entrega  <- dapply[   , list(numero_de_cliente, Predicted) ] #genero la salida

#genero el archivo para Kaggle
fwrite( entrega, file="./kaggle/K101_002.csv", sep="," )